![alt text](files/roofitbasics.png "Title")

<b>RooFit</b> is a OO analysis environment built on ROOT. It is essentially a collection of classes designed to augment root for data modeling whose aim is summarised below (shamelessly stolen from Wouter Verkerke)...

![alt text](files/roofit.png)


We will cover a few of the basics in the session today but note there are many more tutorials available [here](https://root.cern.ch/root/html600/tutorials/roofit/index.html)

In [ ]:
#Setup python in the usual way 
import ROOT
import os
%jsroot on

## RooFit objects

In Roofit, any variable, data point, function, PDF ... is represented by a c++ object

The most basic of these is the `RooRealVar`. Let's create one which will represent the mass of some hypothetical particle, we name it and give it an initial starting value and range.

In [ ]:
MH = ROOT.RooRealVar("MH","mass of the Hypothetical Boson (H-boson) in GeV",125,100,150)
MH.Print()


ok, great. Now we can access this object and modify it. 

In [ ]:
MH.setVal(130)
MH.getVal()

In CMS we don't observe this particle mass but usually define some observable which is *sensitive* to this mass. Lets assume we reconstruct the decay products of the H-boson and measure the invariant mass of those particles. We need to make another variable which represents that invariant mass

In [ ]:
mass = ROOT.RooRealVar("mass","Invariant mass of our decay products in GeV",100,80,200)
mass.Print()

In the perfect world we would perfectly measure the pole mass of the particle in every single event (assuming there is no intrinsic width of the particle). However, CMS is far from perfect so there will be some resolution effect. Lets assume the resolution of our measurement of the invariant mass is 10 GeV and call it "sigma"

In [ ]:
sigma = ROOT.RooRealVar("sigma","Invariant mass resolution in GeV",10,0,20)
sigma.Print()

we can build a "probability denisty function" `f(mass|MH,sigma)` as a simple Gaussian shape...

In [ ]:
gauss = ROOT.RooGaussian("gauss","Gaussian PDF",mass,MH,sigma)
gauss.Print("v")

Notice how the gaussian PDF depends on our `RooRealVar` objects. Its evaluation will depend on their values. Lets plot it for the current values

In [ ]:
plot = mass.frame()
c0 = ROOT.TCanvas()
gauss.plotOn(plot)
plot.Draw()
c0.Draw()

If we change the value of MH, the PDF gets updated too!

In [ ]:
MH.setVal(125)
gauss.plotOn(plot,ROOT.RooFit.LineColor(ROOT.kRed))

MH.setVal(135)
gauss.plotOn(plot,ROOT.RooFit.LineColor(ROOT.kGreen))

plot.Draw()
c0.Update()
c0.Draw()

We can generate MC data from a PDF in RooFit using a single line! Note that we have to tell RooFit which variables to generate in. In this case, each of our events will be a single value of "mass"

In [ ]:
data = gauss.generate(ROOT.RooArgSet(mass),ROOT.RooFit.NumEvents(500))
plot2 = mass.frame()
data.plotOn(plot2)
gauss.plotOn(plot2)
gauss.paramOn(plot2)
c1 = ROOT.TCanvas()
plot2.Draw()
c1.Draw()